# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 3: Algoritmo Genetico para resolver el problema de las n-reinas

Abajo está la implementacion de un algoritmo genético para resolver el problema de las n-reinas. La clase Individual_nqueens implementa el individuo para este problema. Tiene 2 atributos: chromosome y fitness. Cromosome es una lista, donde el elemento i indica la posicion de la reina en la columna i. El fitness es el nro de pares de reinas que no se atacan y se puede obtener llamando a la funcion fitness_nqueens(). La clase individuo también implementa el operador de cruzamiento de un punto (crossover_onepoint), el operador de mutacion de una posicion (mutate_position) y el operador de mutacion swap (mutate_swap). 

La tarea de este laboratorio consiste en implementar un operador de cruzamiento de permutaciones (crossover_permutation) con el fin de que el AG trabaje en el espacio de permutaciones.

Al final de este notebook puede encontrar las tareas y cuestiones que seran evaluadas en este laboratorio.

<b>Clase abstracta de un individuo de algoritmo genético (no modificar)</b>

In [ ]:
class Individual:
    "Clase abstracta para individuos de un algoritmo evolutivo."

    def __init__(self, chromosome):
        self.chromosome = chromosome

    def crossover(self, other):
        "Retorna un nuevo individuo cruzando self y other."
        raise NotImplementedError
        
    def mutate(self):
        "Cambia los valores de algunos genes."
        raise NotImplementedError        

<b>Clase concreta de un individuo del problema de las n-reinas</b> (Falta implementar crossover_uniform y mutate_position)

In [ ]:
import random

class Individual_nqueens(Individual):
    "Clase que implementa el individuo en el problema de las n-reinas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual_nqueens(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_nqueens(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_permutation(self, other):
        "Retorna dos nuevos individuos del cruzamiento de permutacion entre self y other "
        #TODO
        
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual_nqueens(chromosome1)
        ind2 = Individual_nqueens(chromosome2)
        return [ind1, ind2] 

    def mutate_position(self):
        "Cambia aleatoriamente la posicion de una reina."
        mutated_ind = Individual_nqueens(self.chromosome[:])
        indexPos = random.randint(0, len(mutated_ind.chromosome)-1)
        newPos = random.randint(0, len(mutated_ind.chromosome)-1)
        mutated_ind.chromosome[indexPos] = newPos
        return mutated_ind
        
        
    def mutate_swap(self):
        "Intercambia la posicion de dos genes."
        mutated_ind = Individual_nqueens(self.chromosome[:])
        indexOne = random.randint(0,len(mutated_ind.chromosome)-1)
        indexTwo = random.randint(0,len(mutated_ind.chromosome)-1)
        temp = mutated_ind.chromosome[indexOne]
        mutated_ind.chromosome[indexOne] = mutated_ind.chromosome[indexTwo]
        mutated_ind.chromosome[indexTwo] = temp
        return mutated_ind

<b>Funcion de fitness para evaluar un individuo del problema de las n-reinas</b>

In [ ]:
def fitness_nqueens(chromosome):
    """Retorna el fitness de un cromosoma en el problema de las n-reinas (nro de pares de reinas no atacadas) """
    n = len(chromosome)  # nro de reinas
    fitness = 0
    for i in range(n-1):
        for j in range(i+1, n):
            # si el par de reinas i, j  no etan en la misma fila o diagonales => par no atacado
            if chromosome[j] not in [chromosome[i], chromosome[i] - (j-i),  chromosome[i] + (j-i)]:
                fitness = fitness + 1
    return fitness

<b>Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada</b>

In [ ]:
def evaluate_population(population, fitness_fn):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize = len(population)
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_fn(population[i].chromosome)

<b>Funcion que selecciona con el metodo de la ruleta un par de individuos de population para cruzamiento </b>

In [ ]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

<b>Funcion que selecciona sobrevivientes para la sgte generacion, dada la poblacion actual y poblacion de hijos </b>

In [ ]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

<b>Algoritmo Genetico</b>   
Recibe una poblacion inicial, funcion de fitness, numero de generaciones (ngen) y taza de mutación (pmut)

In [ ]:
def genetic_algorithm(population, fitness_fn, ngen=100, pmut=0.1):
    "Algoritmo Genetico "
    
    popsize = len(population)
    evaluate_population(population, fitness_fn)  # evalua la poblacion inicial
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) # cruzamiento 1 punto
            #offspring_population.extend( mating_pool[i][0].crossover_permutation(mating_pool[i][1]) ) # cruzamiento de permutacion

        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                offspring_population[i] = offspring_population[i].mutate_position()   # mutacion de una posicion
                #offspring_population[i] = offspring_population[i].mutate_swap()      # mutacion swap
        
        ## Evalua la poblacion descendencia creada
        evaluate_population(offspring_population, fitness_fn)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
    
    return population[ibest[0]], bestfitness  

 <b>Algoritmo de Busqueda Genetica para el problema de las n-reinas</b>   

In [ ]:
def genetic_search_nqueens(fitness_fn, num_queens=10, popsize=8, ngen=100, pmut=0.1):
    import random
    population = []

    ## Crea la poblacion inicial con cromosomas aleatorios
    #for i in range(popsize):
    #    chromosome = [j for j in range(1,num_queens+1)]
    #    random.shuffle(chromosome)
    #    population.append( Individual_nqueens(chromosome) )
        
    ## Crea la poblacion inicial con los siguientes 6 cromosomas    
    chromosomes = [[1,2,3,4,5,6,7,8,9,10],
                   [5,4,3,2,1,10,9,8,7,6], 
                   [10,9,8,1,2,3,7,6,5,4], 
                   [5,6,7,1,2,3,4,8,9,10], 
                   [2,1,6,7,8,3,4,5,10,9], 
                   [10,7,8,9,6,5,4,1,2,3]] 
                   
    for i in range(popsize):
        population.append( Individual_nqueens(chromosomes[i]) )   
        
    ## llama al algoritmo genetico para encontrar una solucion al problema de las n reinas
    return genetic_algorithm(population, fitness_fn, ngen, pmut)        

###### Tareas y cuestiones:

<b>1) Comparar el desempeño de los operadores de cruzamiento (10 puntos)</b> 

Aqui vamos a comparar el desempeño del operador de cruzamiento <b>crossover_onepoint</b> versus <b>crossover_permutation</b>. Para ello experimentaremos en el problema de 10 reinas con 6 individuos predefinidos y taza de mutación = 0 (a fin de centrarnos solo en el efecto de cruzamiento). Ejecute 10 veces el codigo abajo con crossover_onepoint (registrando el mejor fitness en cada ejecución, junto con la generacion donde se alcanza ese mejor fitness) y luego calcule las medias de los valores registrados. Repita el experimento con crossover_permutation (comente la linea de crossover_onepoint y descomente la linea de crossover_permutation en genetic_algorithm() ). La población inicial está predefinida con el fin de que ambos operadores trabajen a partir de la misma poblacion inicial. De esa forma, solo los operadores de cruzamiento estan influyendo en los resultados. 

In [ ]:
import matplotlib.pyplot as plt
best_ind, bestfitness = genetic_search_nqueens(fitness_nqueens, 10, 6, 100, 0.0)
plt.plot(bestfitness)
plt.show()

<b> - ¿Cual de los dos operadores consigue en media mejores resultados en fitness y en # de generaciones de convergencia? </b> Intente dar una explicación al respecto.       

<b>2) Determinar la influencia de la mutacion  (10 puntos)</b> 

Ahora vamos a identificar la influencia de los operadores de mutación en el AG. Para ello se hará 2 experimentos con las siguientes configuraciones (cada experimento con 10 repeticiones, registrando el mejor fitness y la generacion de convergencia): 1)  operador de cuzamiento crossover_onepoint con el operador de mutacion de una posicion (mutate_position); y 2) operador de cuzamiento crossover_permutation con el operador de mutacion swap (mutate_swap). La taza de mutación a probar es pmut=0.25 (mutacion sucede en aprox. un 25 % de los hijos). 

In [ ]:
best_ind, bestfitness = genetic_search_nqueens(fitness_nqueens, 10, 6, 100, 0.25)
plt.plot(bestfitness)
plt.show()

<b> - ¿Los resultados con los operadores de mutación son mejores que con solo cruzamiento? porque? </b>


<b> - ¿Cual de las dos configuraciones da en media mejores resultados en fitness y # de generaciones para convergir? </b> Intente dar una justificativa al respecto.  

